In [395]:
from Triangulation import *

import torch
import torch.optim as optim



from matplotlib import pyplot as plt

import torch.nn as nn
from torch.autograd import Variable

from  Neural_network import *

%matplotlib qt


In [396]:
def get_set_nb_of_points(point_coordinates):
    set_of_numbers=set()
    for index,_ in enumerate(point_coordinates):
        set_of_numbers.add(len(point_coordinates[index][0]))
    return set_of_numbers


def get_indices_nb_of_points(set_of_numbers,number_of_points,point_coordinates):
    indices=[]
    if number_of_points not in set_of_numbers:
        return "No such number of points for sample"
    else:
        for index,_ in enumerate(point_coordinates):
            if len(point_coordinates[index][0])==number_of_points:
                indices.append(index)
        return indices
    
def get_polygons_nb_of_points(indices):
    
    pass



In [397]:
polygons=load_dataset('12_polygons.pkl')
polygons=[i for i in polygons for j in range(10)]
polygons_reshaped=[]
for polygon in polygons:
    polygons_reshaped.append(polygon.reshape(2,12))

polygons_reshaped=np.array(polygons_reshaped)
#polygons_reshaped=polygons_reshaped.reshape(60000,24)

In [398]:
point_coordinates=load_dataset('12_point_coordinates')
number_of_insertion_points=load_dataset('12_nb_of_points.pkl')
centers_of_mass=load_dataset('12_centers_of_mass')

In [399]:
set_of_points=get_set_nb_of_points(point_coordinates)        
indices=get_indices_nb_of_points(set_of_points,1,point_coordinates)
indices=np.asarray(indices)
number_of_insertion_points=np.array(number_of_insertion_points)

In [400]:
polygons_reshaped.resize(len(point_coordinates),2*12)

polygons_reshaped=np.hstack([polygons_reshaped[indices],number_of_insertion_points[indices,1].reshape(len(indices),1) ])
#polygons_reshaped=polygons_reshaped[indices]


In [401]:
point_coordinates=[ point_coordinates[i][0] for i in indices]
point_coordinates=np.array(point_coordinates)
#barycenters,point_coordinates

In [402]:
#len(polygons_reshaped),len(indices)
#point_coordinates.shape
#centers_of_mass=centers_of_mass.reshape(60000,2)

In [403]:
nb_of_test_data=int(len(polygons_reshaped)*0.2)
nb_of_training_data=int(len(polygons_reshaped)-nb_of_test_data)


In [404]:
x_tensor=torch.from_numpy(polygons_reshaped[:nb_of_training_data]).type(torch.FloatTensor)
x_tensor_test=torch.from_numpy(polygons_reshaped[nb_of_training_data:]).type(torch.FloatTensor)
x_variable,x_variable_test=Variable(x_tensor),Variable(x_tensor_test)

y_tensor=torch.from_numpy(point_coordinates[:nb_of_training_data]).type(torch.FloatTensor)
y_tensor_test=torch.from_numpy(point_coordinates[nb_of_training_data:]).type(torch.FloatTensor)
y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)



shuffle=torch.randperm(x_variable.shape[0])
x_variable = x_variable[shuffle]
y_variable=y_variable[shuffle]

In [405]:
my_net=Net(2*12+1,1*2,nb_of_hidden_layers=3, nb_of_hidden_nodes=20,batch_normalization=True)

print("Training data length:",x_variable.size()[0])

Training data length: 174978


In [406]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4,weight_decay=0)
#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-5,weight_decay=.5,momentum=0.9)
#max_distance=0.6108970818704328
loss_func =torch.nn.MSELoss(size_average=False) 
#loss_func=myOtherLossfunction()


In [407]:
#if  torch.cuda.is_available():
#    loss_func
#    x_variable , y_variable=x_variable.cuda(), y_variable.cuda()
#    x_variable_test,y_variable_test= Variable(x_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)
#    print("cuda activated")


In [ ]:
batch_size=int(x_variable.size()[0]/18)
nb_of_epochs=13000
#my_net.cuda()

# Train the network #
my_net.train()
for t in range(nb_of_epochs):
    sum_loss1=0
    sum_loss2=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss2= loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size).resize(batch_size,2))
        
        #loss1=my_torch_loss_function(out, y_variable.narrow(0,b,batch_size).resize(batch_size,1*2)).sum()
        #sum_loss1+=loss1.data[0]
        sum_loss2+=loss2.data[0]

        
        optimizer.zero_grad()   # clear gradients for next train
        loss2.sum().backward()         # backpropagation, compute gradients
        #print(t,loss1.data[0],loss2.data[0])
        optimizer.step()        # apply gradients
    if t%10==0: 
        my_net.eval()
        test_loss=loss_func(my_net(x_variable_test),y_variable_test).data[0]
        my_net.train()
        print("Epoch:",t,"Training Loss:",sum_loss2/(x_variable.size(0)),"Test Loss:",test_loss/x_variable_test.size(0))
        #print("Epoch:",t,"Training Loss:",sum_loss1/(x_variable.size(0)),sum_loss2/(x_variable.size(0)))

Epoch: 0 Training Loss: 0.2615481900537139 Test Loss: 0.24979037303687363
Epoch: 10 Training Loss: 0.16236317364718708 Test Loss: 0.15743294124979998
Epoch: 20 Training Loss: 0.11650238527196098 Test Loss: 0.11463449362341122
Epoch: 30 Training Loss: 0.09511936857776407 Test Loss: 0.09484516135427144
Epoch: 40 Training Loss: 0.08506720853277805 Test Loss: 0.08586102187764322
Epoch: 50 Training Loss: 0.07837054984233732 Test Loss: 0.0796115989657153
Epoch: 60 Training Loss: 0.07266551239488411 Test Loss: 0.07401284564561711
Epoch: 70 Training Loss: 0.06768034083548181 Test Loss: 0.06901260074595944
Epoch: 80 Training Loss: 0.06362087123876824 Test Loss: 0.06473227042565838
Epoch: 90 Training Loss: 0.06042328175609041 Test Loss: 0.06149087723102025
Epoch: 100 Training Loss: 0.05788156853223155 Test Loss: 0.05915265477479768
Epoch: 110 Training Loss: 0.05580364896542351 Test Loss: 0.057332829305876236
Epoch: 120 Training Loss: 0.05405799023244395 Test Loss: 0.05584647590955331
Epoch: 130 

Epoch: 1060 Training Loss: 0.038888406545885414 Test Loss: 0.045064031772571664
Epoch: 1070 Training Loss: 0.03886259066095095 Test Loss: 0.045051181237891195
Epoch: 1080 Training Loss: 0.03883731212733038 Test Loss: 0.04505067893686134
Epoch: 1090 Training Loss: 0.038813678785187905 Test Loss: 0.045050341278946825
Epoch: 1100 Training Loss: 0.03879024583793659 Test Loss: 0.04504919435826199
Epoch: 1110 Training Loss: 0.038767845735372355 Test Loss: 0.04503999666829308
Epoch: 1120 Training Loss: 0.038746411504790065 Test Loss: 0.045034979239117076
Epoch: 1130 Training Loss: 0.03872657694517534 Test Loss: 0.04503139894899872
Epoch: 1140 Training Loss: 0.03870656396608833 Test Loss: 0.04502000787786611
Epoch: 1150 Training Loss: 0.038687157055101215 Test Loss: 0.04499555977051853
Epoch: 1160 Training Loss: 0.038669363767728566 Test Loss: 0.04497452451961269
Epoch: 1170 Training Loss: 0.038651127483816716 Test Loss: 0.04495062894339795
Epoch: 1180 Training Loss: 0.03863329512903274 Test L

Epoch: 2110 Training Loss: 0.0375951873291303 Test Loss: 0.04479989677602214
Epoch: 2120 Training Loss: 0.037589138333473084 Test Loss: 0.044805871367716146
Epoch: 2130 Training Loss: 0.03758348506976368 Test Loss: 0.044816009476835394
Epoch: 2140 Training Loss: 0.03757739334412531 Test Loss: 0.04481394446149043
Epoch: 2150 Training Loss: 0.0375712526098147 Test Loss: 0.04481123761705177
Epoch: 2160 Training Loss: 0.03756530163150198 Test Loss: 0.04481233151707234
Epoch: 2170 Training Loss: 0.03755925228702858 Test Loss: 0.04480565649449782
Epoch: 2180 Training Loss: 0.0375535020524232 Test Loss: 0.04479997491173789
Epoch: 2190 Training Loss: 0.03754802685936991 Test Loss: 0.04479085535748474
Epoch: 2200 Training Loss: 0.03754213971441678 Test Loss: 0.04478761830640345
Epoch: 2210 Training Loss: 0.037536175306681414 Test Loss: 0.04478853919162485
Epoch: 2220 Training Loss: 0.03752975883318635 Test Loss: 0.0447933082608472
Epoch: 2230 Training Loss: 0.03752262501923233 Test Loss: 0.0447

Epoch: 3160 Training Loss: 0.037181785923679066 Test Loss: 0.045048072552628646
Epoch: 3170 Training Loss: 0.03718050664036604 Test Loss: 0.04504664657581611
Epoch: 3180 Training Loss: 0.03717921968309722 Test Loss: 0.04504545221558957
Epoch: 3190 Training Loss: 0.0371779606311222 Test Loss: 0.04504090918183065
Epoch: 3200 Training Loss: 0.03717717387624489 Test Loss: 0.04505180911417852
Epoch: 3210 Training Loss: 0.03717522172653491 Test Loss: 0.04505262395807139
Epoch: 3220 Training Loss: 0.037173545664825544 Test Loss: 0.04506476569018751
Epoch: 3230 Training Loss: 0.03717224004589148 Test Loss: 0.045059954762546005
Epoch: 3240 Training Loss: 0.037170917858189226 Test Loss: 0.04506487452207732
Epoch: 3250 Training Loss: 0.03716991727899811 Test Loss: 0.0450686334081174
Epoch: 3260 Training Loss: 0.0371690070431957 Test Loss: 0.045065639135867204
Epoch: 3270 Training Loss: 0.037167576548071846 Test Loss: 0.045073756878621925
Epoch: 3280 Training Loss: 0.03716729034440223 Test Loss: 0

Epoch: 4210 Training Loss: 0.03700846998081321 Test Loss: 0.045364851487668306
Epoch: 4220 Training Loss: 0.03700691792325321 Test Loss: 0.04536958148903278
Epoch: 4230 Training Loss: 0.037005285637973506 Test Loss: 0.04535775229977968
Epoch: 4240 Training Loss: 0.03700403042297639 Test Loss: 0.04536655652060854
Epoch: 4250 Training Loss: 0.03700279561372537 Test Loss: 0.04536803272752406
Epoch: 4260 Training Loss: 0.037001495401441986 Test Loss: 0.04537019541251371
Epoch: 4270 Training Loss: 0.036999609875620974 Test Loss: 0.045367974125737245
Epoch: 4280 Training Loss: 0.03699880184295713 Test Loss: 0.04537864244149912
Epoch: 4290 Training Loss: 0.036997039798061224 Test Loss: 0.04538559652020134
Epoch: 4300 Training Loss: 0.03699562639492983 Test Loss: 0.045384357510994365
Epoch: 4310 Training Loss: 0.036995026431112944 Test Loss: 0.04538600394214778
Epoch: 4320 Training Loss: 0.036993764065384294 Test Loss: 0.0453863164850108
Epoch: 4330 Training Loss: 0.03699237455616072 Test Loss

Epoch: 5260 Training Loss: 0.03688924007840673 Test Loss: 0.045575552816886314
Epoch: 5270 Training Loss: 0.036888509482933214 Test Loss: 0.04556878849635093
Epoch: 5280 Training Loss: 0.03688750995019061 Test Loss: 0.04557082560608312
Epoch: 5290 Training Loss: 0.03688698410981041 Test Loss: 0.045576230923276624
Epoch: 5300 Training Loss: 0.03688572697632435 Test Loss: 0.045572653423719536
Epoch: 5310 Training Loss: 0.03688516032445195 Test Loss: 0.045576875542931605
Epoch: 5320 Training Loss: 0.036884032427357834 Test Loss: 0.0455677252925044
Epoch: 5330 Training Loss: 0.03688366948413024 Test Loss: 0.04557201717574839
Epoch: 5340 Training Loss: 0.03688290644875267 Test Loss: 0.045562702282205846
Epoch: 5350 Training Loss: 0.03688175308807795 Test Loss: 0.04556262414649009
Epoch: 5360 Training Loss: 0.036880964937935946 Test Loss: 0.045568470372365354
Epoch: 5370 Training Loss: 0.03688042723780588 Test Loss: 0.04557270365382252
Epoch: 5380 Training Loss: 0.03687964152937709 Test Loss

Epoch: 6310 Training Loss: 0.03679406558571158 Test Loss: 0.045627568879089135
Epoch: 6320 Training Loss: 0.0367936604357271 Test Loss: 0.04562305654150426
Epoch: 6330 Training Loss: 0.03679333847839979 Test Loss: 0.04561962136057231
Epoch: 6340 Training Loss: 0.03679248178587987 Test Loss: 0.04561588200846116
Epoch: 6350 Training Loss: 0.036792021174123946 Test Loss: 0.04561189708695764
Epoch: 6360 Training Loss: 0.036791727645314694 Test Loss: 0.04560939116293092
Epoch: 6370 Training Loss: 0.036791079196049827 Test Loss: 0.04561051575912554
Epoch: 6380 Training Loss: 0.036790010074480795 Test Loss: 0.04560722568737998
Epoch: 6390 Training Loss: 0.03678969282617181 Test Loss: 0.04560457186360558
Epoch: 6400 Training Loss: 0.03678935220717926 Test Loss: 0.045599197242586126
Epoch: 6410 Training Loss: 0.036788823227453506 Test Loss: 0.04560758566978471
Epoch: 6420 Training Loss: 0.036788876945144086 Test Loss: 0.045607730778971116
Epoch: 6430 Training Loss: 0.036788584986007615 Test Los

Epoch: 7360 Training Loss: 0.03673384038061436 Test Loss: 0.04567841569611689


In [382]:
sample_index=7
sample_contour=np.delete(polygons_reshaped[sample_index],24)
sample_contour=sample_contour.reshape(12,2)

nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length(sample_contour,.8)
inserted_point_coordinates=np.array(inserted_point_coordinates)

my_net.eval()
predictions=my_net(x_variable).cpu()
predicted_inserted_points=predictions.data.numpy()[sample_index]
predicted_inserted_points=predicted_inserted_points.reshape(1,2)


plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='Predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')
#plt.scatter(polygons_reshaped[3112][0::2].sum()/12,polygons_reshaped[312][1::2].sum()/12,label='Original points')

plot_contour(sample_contour)
plt.legend()


#original_points=point_coordinates.reshape(218722,2)
#plt.scatter(original_points[:,0],original_points[:,1],label='Original points')



13 0.00646368949978321 -0.1199108451461814 0


In [373]:

#torch.cuda.empty_cache()

Net(
  (bn_input): BatchNorm1d(25, eps=1e-05, momentum=0.5, affine=True)
  (fc0): Linear(in_features=25, out_features=20, bias=True)
  (bn0): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
  (predict): Linear(in_features=20, out_features=2, bias=True)
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.5, affine=True)
)

distances=[np.linalg.norm(i-j) for i in centers_of_mass for j in predicted_inserted_points]
distances=np.array(distances)
100*distances.sum()/(x_variable.size(0) *max_distance)

In [383]:
x_variable,x_variable_test=x_variable.resize(x_variable.size()[0],1,25),Variable(x_tensor_test.view(x_variable_test.size()[0],1,25),volatile=True)

In [389]:
# Using convolution network
# O: output dimension
# I: Input dimensiion
# S: Stride
# P: padding
# w: kernel size
# O=(I-w-2*P)/S+1


nb_of_points_output=1

class Conv_net(nn.Module):
    
    def __init__(self):
        super(Conv_net,self).__init__()
        self.conv1=nn.Conv1d(1,14,kernel_size=14,stride=1)
        self.conv2=nn.Conv1d(14,28,kernel_size=2,stride=1)

        self.bn1=nn.BatchNorm1d(num_features=28*9)
        self.fc1=nn.Linear(28*9,12)
        self.dp_1=nn.Dropout(p=0.5)

        
        self.bn2=nn.BatchNorm1d(num_features=12)
        self.fc2=nn.Linear(12,12)
        self.dp_2=nn.Dropout(p=0.5)

        
        self.bn3=nn.BatchNorm1d(num_features=12)
        self.fc3=nn.Linear(12,12)
        self.dp_3=nn.Dropout(p=0.5)
        
        self.bn4=nn.BatchNorm1d(num_features=12)
        self.fc4=nn.Linear(12,nb_of_points_output*2)
        
    def forward(self,x):
        x=F.relu(F.max_pool1d(self.conv1(x),kernel_size=2,stride=1))
        
        #x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        #x=self.conv2(x)

        x=F.relu(F.max_pool1d(self.conv2(x),kernel_size=2,stride=1))
        x=F.relu(self.fc1(self.bn1(x.view(-1,28*9))))
        x=F.relu(self.fc2( self.dp_1(self.bn2(x))))
        x=F.relu(self.fc3(self.dp_2(self.bn3(x))))

        x=self.fc4(self.bn4(self.dp_3(x)))

        return x
    
        

In [394]:
#torch.cuda.empty_cache()
my_conv_net=Conv_net()
print(my_conv_net)

RuntimeError: cuda runtime error (77) : an illegal memory access was encountered at torch/csrc/cuda/Module.cpp:321

In [391]:
optimizer = torch.optim.Adam(my_conv_net.parameters(), lr=1e-4,weight_decay=0.5)

#optimizer = torch.optim.SGD(my_net.parameters(), lr=1e-5,weight_decay=.5,momentum=0.9)

loss_func = torch.nn.MSELoss(size_average=False) 
#loss_func=torch.nn.SmoothL1Loss()


In [392]:
if  torch.cuda.is_available():
    loss_func.cuda()
    my_conv_net.cuda()
    x_variable , y_variable,x_variable_test,y_variable_test= x_variable.cuda(), y_variable.cuda(),x_variable_test.cuda(),y_variable_test.cuda()
    print("cuda activated")


cuda activated


In [393]:

batch_size=int(x_variable.size()[0]/2 )
nb_of_epochs=13000
# Train the network #
my_conv_net.train()
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_conv_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss = loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        #loss=loss_func.apply(out, y_variable.narrow(0,b,batch_size))
        #loss=torch.sqrt((out[0]-y_variable.narrow(0,b,batch_size)[0]).pow(2)+(out[1]-y_variable.narrow(0,b,batch_size)[1]).pow(2)) 
        sum_loss+=loss.data[0]
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        #print(t,loss.data[0])
        optimizer.step()        # apply gradients
    if t%10==0:
        my_conv_net.eval()
        test_loss=loss_func(my_conv_net(x_variable_test),y_variable_test).data[0]
        my_conv_net.train()
        print("Epoch:",t,"Training Loss:",sum_loss/x_variable.size(0),"Test Loss:",test_loss/x_variable_test.size(0))

RuntimeError: reduce failed to synchronize: an illegal memory access was encountered

In [163]:
x_variable=x_variable.cpu()
my_conv_net=my_conv_net.cpu()

In [164]:
contour1=np.delete(polygons_reshaped[0],24)
contour1=contour1.reshape(12,2)
plot_contour(contour1)

In [169]:
contour1=np.delete(polygons_reshaped[11],24)
contour1=contour1.reshape(12,2)
nb_of_inserted_points,inserted_point_coordinates=get_extrapoints_target_length(contour1,.8
                                                                              )
inserted_point_coordinates=np.array(inserted_point_coordinates)
my_conv_net.eval()
predictions=my_conv_net(x_variable).cpu()
predicted_inserted_points=predictions.data.numpy()[11]
predicted_inserted_points=predicted_inserted_points.reshape(2,2)
plt.scatter(predicted_inserted_points[:,0],predicted_inserted_points[:,1],label='Predicted points')
plt.scatter(inserted_point_coordinates[:,0],inserted_point_coordinates[:,1],label='Original points')

plot_contour(contour1)
plt.legend()

13 -0.3033108530139085 0.1448011230082973 0
14 0.4449916807636812 0.4447385301615993 0


In [119]:
contour=polygons_reshaped[1]
contour

array([ 0.72589751,  0.17159901,  0.84207899,  0.40510667,  0.71337615,
        0.63783057,  0.16721335,  0.58156785, -0.53593486,  0.85478697,
       -1.19396456,  0.50564378, -0.99666214,  0.37817729, -0.86514855,
       -0.73118862, -0.06646145, -0.53079973,  0.05744332, -1.1159677 ,
        0.52287723, -0.86178872,  0.62928501, -0.29496737,  0.5       ])

In [45]:
d_tensor=torch.FloatTensor([[2.2,1.2],[3,3]])
a_tensor=torch.FloatTensor([[1,2],[3,2]])


In [46]:
d_variable=Variable(d_tensor)

a_variable=Variable(a_tensor)




In [22]:
d_variable.requires_grad=False
a_variable.requires_grad=True

In [184]:
#fun=torch.sqrt((a_variable[0]-d_variable[0]).pow(2)+(a_variable[1]-d_variable[1]).pow(2))
#print(fun)

def my_torch_loss_function(a,b)->Variable:
    torch_sum=0
    for i in range(0,a.size()[1],2):
        euclidean_distance=torch.sqrt((a[:,i]-b[:,i]).pow(2)+(a[:,i+1]-b[:,i+1]).pow(2))
        torch_sum+=euclidean_distance
    return  torch_sum   
    
        
    
loss_result=my_torch_loss_function(a_variable,d_variable) 

loss_result.sum()

Variable containing:
 2.4422
[torch.FloatTensor of size 1]

In [248]:
loss_result.sum().backward()


In [249]:
a_variable.grad

Variable containing:
-2.4000  1.6000
 0.0000 -2.0000
[torch.FloatTensor of size 2x2]

In [254]:
another_loss_func=torch.nn.MSELoss(size_average=False)
another_loss_result=another_loss_func(out,y_variable.view(0,b,batch_size))

another_loss_result


RuntimeError: invalid argument 2: size '[0 x 0 x 87489]' is invalid for input with 349956 elements at C:\Anaconda2\conda-bld\pytorch_1519501749874\work\torch\lib\TH\THStorage.c:41

In [170]:
from torch.autograd.function import Function
from math import pow

class myLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target):
        self.save_for_backward(output,target) 

                        
       # output=output.view(int(output.size()[0]/2),2)

        #target=target.view(int(target.size()[0]/2),2)
        distance=torch.nn.PairwiseDistance()
        result=distance(output,target)

        result=torch.FloatTensor(result)
        #self.save_for_backward(result)


        return  result 
    
    
    @staticmethod
    def backward(self,grad_output1):
        input1,target=self.saved_variables
        
        print(input1)
        #distance=torch.nn.PairwiseDistance()(input1.view(int(input1.size()[0]/2),2),target.view(int(target.size()[0]/2),2))
        distance=torch.nn.PairwiseDistance()(input1,target)

        grad_output1=(input1-target)/distance

        
        return grad_output1,None
    
    
class myOtherLossfunction(Function):
    
    @staticmethod
    def forward(self,output,target)->Variable:
        
        torch_sum=0
        for i in range(0,output.size()[1],2):
            euclidean_distance=(output[:,i]-target[:,i]).pow(2)+(output[:,i+1]-target[:,i+1]).pow(2)
            torch_sum+=euclidean_distance
        return  torch_sum


    
    

In [99]:
myloss=myLossfunction().apply(out,y_variable.narrow(0,b,batch_size).resize(batch_size,2))
myloss.sum().backward()

Variable containing:
-3.7984e-01  2.5756e-01
 3.1686e-02  1.0344e-01
-1.7014e-01 -1.0829e-02
           ⋮            
-1.2210e-01 -1.6554e-01
-2.6152e-02 -1.0011e-01
-1.9689e-01 -1.8628e-01
[torch.FloatTensor of size 87489x2]



RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [180]:
# Flush all variables in GPU
del my_conv_net,my_net,x_variable_test,x_variable,y_variable,y_variable_test,loss_func
torch.cuda.empty_cache()


NameError: name 'my_conv_net' is not defined

In [181]:
'''
Should I order the interior points ?

For each interior point calculate distance with points of the polygon

find which ti which point it is closer and sort it according to the index.

If multiple points are closer to the same point of the contour then take into

account the distance to the point.

'''

'\nShould I order the interior points ?\n\nFor each interior point calculate distance with points of the polygon\n\nfind which ti which point it is closer and sort it according to the index.\n\nIf multiple points are closer to the same point of the contour then take into\n\naccount the distance to the point.\n\n'

In [193]:
def get_er(n,r)->'joules':
    return n*r

In [194]:
get_er.__annotations__

{'return': 'joules'}

In [199]:
print("{:,},{}".format(get_er(4,5)))

IndexError: tuple index out of range

In [33]:
out.size()[1]

2

In [31]:
y_variable

Variable containing:
(  0   ,.,.) = 
 -0.0732 -0.0308

(  1   ,.,.) = 
 -0.1070 -0.2541

(  2   ,.,.) = 
 -0.2270  0.0384
  ...  

(174975,.,.) = 
 -0.0972 -0.0611

(174976,.,.) = 
  0.1325  0.0653

(174977,.,.) = 
 -0.2360  0.0878
[torch.FloatTensor of size 174978x1x2]

In [129]:
ewew=my_torch_loss_function(out,y_variable.narrow(0,b,batch_size).resize(batch_size,2))
ewew.sum()

Variable containing:
 14048.2607
[torch.FloatTensor of size 1]

In [127]:
loss=nn.MSELoss(size_average=False)

In [128]:
loss(out,y_variable.narrow(0,b,batch_size))

Variable containing:
 14047.7451
[torch.FloatTensor of size 1]

AttributeError: 'MSELoss' object has no attribute 'backward'